In [12]:
%%html
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML
def printmd(string):
    from IPython.display import Markdown, display
    display(Markdown(string))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### postgresql + python ###
import psycopg2 as pg2

class psql_py:
    def __init__(self):
        self.conn = self.connect_db()
        self.cur = self.conn.cursor()
        self.cur.execute("set search_path = mimiciii;") 
        
    def connect_db(self):
        try:
            conn = pg2.connect("host = localhost dbname=mimic user=postgres port=5432")
            #print('database connected')
            return conn
        except:
            print("I am unable to connect to the database")          
            
    def exe_sql (self, exe_sql_str):
        return pd.read_sql(exe_sql_str,self.conn)
    
    def db_query(self, exe_sql_str, mode='sql'):
        if mode == 'pd':
            return_pd = self.exe_sql (exe_sql_str)
            self.close_db()
            return return_pd
        else: 
            self.cur.execute(exe_sql_str)
            self.close_db()
            
    def close_db (self):
        self.cur.close()
        # commit the changes
        self.conn.commit()
        #print('database disconnected')


In [3]:
db = psql_py() 
db.cur.execute(open('./sql/ventilation-durations_mod.sql','r').read())
print('execution complete: ventilation-durations_mod.sql')
db.cur.execute(open('./sql/ventduration_on_off_time.sql','r').read())
print('execution complete: ventduration_on_off_time.sql')
db.cur.execute(open('./sql/ptsinfo_vent.sql','r').read())
print('execution complete: ptsinfo_vent.sql')

str_sql = 'select * from ventdurations'
temp = db.db_query(str_sql,'pd')
total = len(temp.icustay_id.unique())
printmd('## Population using a ventilation extracted by [code](https://github.com/yyjo0430/mimiciii_extubation/tree/master/sql/ventilation-durations_mod.sql): '
        + str(total)+ ' patients \n'+
        '- ### Based on [mimic-code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation-durations.sql) \n' )




execution complete: ventilation-durations_mod.sql
execution complete: ventduration_on_off_time.sql
execution complete: ptsinfo_vent.sql


## Population using a ventilation extracted by [code](https://github.com/yyjo0430/mimiciii_extubation/tree/master/sql/ventilation-durations_mod.sql): 26801 patients 
- ### Based on [mimic-code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation-durations.sql) 


In [4]:
db = psql_py() 
str_sql = 'select * from ptsinfo_vent'
case_missing = db.db_query(str_sql,'pd')
case_missing = case_missing.sort_values(['icustay_id','ventnum','vent_on_time','vent_off_time']).reset_index(drop=True)
case_missing = case_missing.dropna(subset=['icustay_id'])
case_missing.icustay_id = case_missing.icustay_id.astype(int)
num_case_missing = len(case_missing.icustay_id.unique())

printmd('### Exclusion 1 - Population: '+str(num_case_missing) + ', excluded '+ str(total-num_case_missing)+' patients \n' +
'- Excluded patient with records with missing values in variables such as extubation, self-extubation, oxygentherapy, and so on'      
)
display(case_missing.sample(5))


### Exclusion 1 - Population: 21557, excluded 5244 patients 
- Excluded patient with records with missing values in variables such as extubation, self-extubation, oxygentherapy, and so on

,icustay_id,hadm_id,subject_id,gender,birth_time,extu_age,admittime,ventnum,vent_on_time,vent_off_time,duration_hours,unplanned,tra_used,trache_time,dischtime,death_time,death
13841,247391,161187,68916,F,2085-02-15,85.79,2170-11-29 18:16:00,1,2170-11-30 15:00:00,2170-12-08 14:00:00,191.000000,0,0,NaT,2170-12-25 17:15:00,NaT,0
17066,258315,172892,23097,M,2107-08-22,77.34,2184-11-29 19:15:00,2,2184-12-23 09:07:00,2184-12-28 08:30:00,119.383333,0,0,2185-01-03 20:00:00,2185-01-21 12:00:00,2185-01-29,0
16290,255740,133707,11504,F,2140-09-22,0.02,2140-09-22 07:27:00,3,2140-09-29 17:00:00,2140-10-08 13:00:00,212.000000,0,0,2140-10-12 19:00:00,2140-12-21 13:30:00,NaT,0
21599,273640,132869,22220,F,2136-12-01,0.09,2136-12-01 08:24:00,3,2137-01-04 11:00:00,2137-01-07 08:00:00,69.000000,0,0,2136-12-08 20:00:00,2137-02-24 18:40:00,NaT,0
18503,263042,187326,19082,F,1869-10-30,300.00,2169-10-30 04:28:00,1,2169-10-30 05:00:00,2169-11-02 10:30:00,77.500000,0,0,NaT,2169-11-06 19:45:00,2169-11-10,0


In [5]:
case_adult = case_missing.copy()
case_adult = case_adult[(case_adult.extu_age>15 )& (case_adult.extu_age<90)].sort_values('icustay_id')
num_case_adult = len(case_adult.icustay_id.unique())
printmd('### Exclusion 2 - Population: '+ str(num_case_adult) + ', excluded '+str(num_case_missing-num_case_adult)+' patients '+ '\n' +
'- Only adult: >15 & <89  \n'
)



### Exclusion 2 - Population: 20182, excluded 1375 patients 
- Only adult: >15 & <89  


In [6]:
case_unplan = case_adult.copy()
case_unplan = case_unplan[case_unplan.unplanned==0]
num_case_unplan = len(case_unplan.icustay_id.unique())

printmd('### Exclusion 3 - Population: '+ str(num_case_unplan) + ', excluded '+str(num_case_adult-num_case_unplan)+'  patients \n' + 
'- Excluded the unplanned extubation cases')


### Exclusion 3 - Population: 19995, excluded 187  patients 
- Excluded the unplanned extubation cases

In [7]:
case_tra = case_unplan.copy()
case_tra = case_tra[case_tra.tra_used==0]
num_case_tra = len(case_tra.icustay_id.unique())

printmd('### Exclusion 4 - Population: '+ str(num_case_tra) + ', excluded '+str(num_case_unplan-num_case_tra)+' patients \n' +
'- Excluded patient using the tracheostomy')



### Exclusion 4 - Population: 19806, excluded 189 patients 
- Excluded patient using the tracheostomy

In [8]:
case_death = case_tra.copy()
case_death = case_death[case_death.death==0]
num_case_death = len(case_death.icustay_id.unique())
printmd('### Exclusion 5 - Population: '+ str(num_case_death) + ', excluded '+str(num_case_tra-num_case_death)+' patients \n' +
'- Excluded the dead'    
)


### Exclusion 5 - Population: 19296, excluded 510 patients 
- Excluded the dead

In [9]:
case_period = case_death.copy()
case_period = case_period[(case_period.duration_hours>1)]
num_case_period = len(case_period.icustay_id.unique())
printmd('### Exclusion 6 - Population: '+ str(num_case_period) + ', excluded '+str(num_case_death-num_case_period)+' patients\n' +
'- Excluded patients with short period (<1 hour) for the use of the mechanical ventilation'    
)


### Exclusion 6 - Population: 18916, excluded 380 patients
- Excluded patients with short period (<1 hour) for the use of the mechanical ventilation

In [10]:
case_ventnum = case_period.groupby(['hadm_id','icustay_id']).filter(lambda x:x['ventnum'].min()==1)
case_ventnum = case_ventnum.drop(case_ventnum[case_ventnum['ventnum'] >2]['icustay_id'].index)
num_case_ventnum = len(case_ventnum.icustay_id.unique())
printmd('### Exclusion 7 - Population: '+ str(num_case_ventnum) + ', excluded '+str(num_case_period-num_case_ventnum)+' patients \n' +
        '- Excluded patients whose starting ventnum is not 1 (missing for the trial to the first macheical vailtiation) '+
        'and is greater than 2 (multiple trials to use the macheical vailtiation) \n' +
        '\t - The ventilation number (*ventnum*) means the sequence number of use of the mechanical ventilation \n' +    
        '\t\t - The each single mechanical ventilation is defined as a set of records where the interval time between adjacent two records is not over 8 hours'
)


### Exclusion 7 - Population: 17686, excluded 1230 patients 
- Excluded patients whose starting ventnum is not 1 (missing for the trial to the first macheical vailtiation) and is greater than 2 (multiple trials to use the macheical vailtiation) 
	 - The ventilation number (*ventnum*) means the sequence number of use of the mechanical ventilation 
		 - The each single mechanical ventilation is defined as a set of records where the interval time between adjacent two records is not over 8 hours

In [11]:
printmd('## Outcome: the extubation failure')
printmd(' - Re-intubation within 12 hours')
temp_id = (case_ventnum.groupby(['icustay_id'])['ventnum'].count()>1) ## > 2 trials 
temp_id = temp_id[temp_id==True].index
out_cand = case_ventnum[case_ventnum['icustay_id'].isin(temp_id)].sort_values(['icustay_id','ventnum'])
threshold_id = out_cand.groupby(['icustay_id']).apply(lambda x : (x['vent_on_time'].iloc[1] - x['vent_off_time'].iloc[0])
                                      /np.timedelta64(1,'h')
                                      )>12 ## threshold
outcome_ids = threshold_id[threshold_id==True].index
case_outcome= case_ventnum.copy()

case_outcome['outcome'] = 0
case_outcome['outcome'][(case_outcome['icustay_id'].isin(outcome_ids)) 
                       #&(case_outcome['ventnum']==1)
                       ]=1
printmd(' Control: '+str(len(set(case_outcome['icustay_id'][case_outcome['outcome']==0])))+'\n'
        '- Case: '+str(len(set(case_outcome['icustay_id'][case_outcome['outcome']==1]))) +'\n'+
        '\t - Patients with 2 more trials '+str(len(set(temp_id)))
       )



### Outcome: the extubatin failure

 - Re-intubation within 12 hours

- Control: 16716
- Case: 970
	 - Patients with 2 more trials 1881